<a href="https://colab.research.google.com/github/Nikkiie/Bike-sharing-demand-prediction-Cap-2-/blob/main/Individual_colab_Bike_sharing_demand_prediction_(Capstone_2)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PROJECT TITLE - Seoul Bike Sharing Demand Prediction** 

# **PROJECT SUMMARY:**
 
 **Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.**

 # **Data Description:**

 **Attribute Information:**

  --> Date : year-month-day

  --> Rented Bike count - Count of bikes rented at each hour

  --> Hour - Hour of he day

  --> Temperature-Temperature in Celsius

  --> Humidity - %

  --> Windspeed - m/s

  --> Visibility - 10m

  --> Dew point temperature - Celsius

  --> Solar radiation - MJ/m2

  --> Rainfall - mm

  --> Snowfall - cm

  --> Seasons - Winter, Spring, Summer, Autumn

  --> Holiday - Holiday/No holiday

  --> Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

# **GitHub Link -**

### Importing Libraries

In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import Lasso,Ridge
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

### Loading the dataset

In [2]:
#Mounting the drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Giving the file path

file_path = ('/content/drive/MyDrive/cap 2/SeoulBikeData (1).csv')
bike_df = pd.read_csv(file_path, encoding = 'ISO-8859-1')

### Dataset First View


In [7]:
# It is a good habit to copy the data for backup
df = bike_df.copy()

In [ ]:
# Checking top 5 rows in dataframe
bike_df.head()

In [ ]:
# Checking last 5 rows in dataframe
bike_df.tail()

### Dataset Rows & Columns count

In [10]:
# Data shape
bike_df.shape

(8760, 14)

In [11]:
#Check all the coloumns present in our dataset
bike_df.columns

Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day'],
      dtype='object')

In [12]:
# Renaming Columns
bike_df.rename(columns={'Date': 'Date', 'Rented Bike Count': 'bike_count', 'Hour': 'hour',
                   'Temperature(°C)': 'temp', 'Humidity(%)': 'humidity', 'Wind speed (m/s)': 'wind_speed',
                   'Visibility (10m)': 'visibility', 'Dew point temperature(°C)': 'dew_temp',
                   'Solar Radiation (MJ/m2)': 'sunlight', 'Rainfall(mm)': 'rain', 'Snowfall (cm)': 'snow',
                   'Seasons': 'season', 'Holiday': 'holiday', 'Functioning Day': 'functioning_day'}, inplace=True)

In [ ]:
# again checking head of dataframe

bike_df.head()

### Dataset Information

In [14]:
# data dtype
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             8760 non-null   object 
 1   bike_count       8760 non-null   int64  
 2   hour             8760 non-null   int64  
 3   temp             8760 non-null   float64
 4   humidity         8760 non-null   int64  
 5   wind_speed       8760 non-null   float64
 6   visibility       8760 non-null   int64  
 7   dew_temp         8760 non-null   float64
 8   sunlight         8760 non-null   float64
 9   rain             8760 non-null   float64
 10  snow             8760 non-null   float64
 11  season           8760 non-null   object 
 12  holiday          8760 non-null   object 
 13  functioning_day  8760 non-null   object 
dtypes: float64(6), int64(4), object(4)
memory usage: 958.2+ KB


#### Duplicate Values

In [16]:
# Checking duplicates
print(len(bike_df[df.duplicated()]))

0


#### Missing Values/Null Values

In [17]:
# Missing data counts
bike_df.isnull().sum()

Date               0
bike_count         0
hour               0
temp               0
humidity           0
wind_speed         0
visibility         0
dew_temp           0
sunlight           0
rain               0
snow               0
season             0
holiday            0
functioning_day    0
dtype: int64